In [1]:
!pip install --quiet -U langgraph

**States** are the first things defined about the graph. These consist of the schema of the graph, and reducer functions to specify how to apply state updates.

In [2]:
from typing_extensions import TypedDict

class State(TypedDict):
    graph_state: str

**Nodes** are python functions (synchronous or asynchronous). The first positional argument is the state.

In [4]:
def node_1(state):
    print("---Node 1---")
    return {"graph state": state['graph_state'] + " I am"}

def node_2(state):
    print("---Node 2---")
    return {"graph state": state['graph_state'] + " happy!"}

def node_3(state):
    print("---Node 3---")
    return {"graph state": state['graph_state'] + " sad!"}

**Schemas** are typically constructed in `TypedDict` types. `Pydantic` base models can also be used.

In [3]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

class InputState(TypedDict):
    question: str

class OutputState(TypedDict):
    answer: str

class OverallState(InputState, OutputState):
    pass

def answer_node(state: InputState):
    return {"answer": "bye", "question": state["question"]}

builder = StateGraph(OverallState, input=InputState, output=OutputState)
builder.add_node(answer_node)
builder.add_edge(START, "answer_node")
builder.add_edge("answer_node", END)
graph = builder.compile()

print(graph.invoke({"question": "hi"}))

{'answer': 'bye'}


**Edges** connect each node. Normal edges are used if you want to always go from like `node_1` to `node_2`.
**Conditional edges** are used if you want optional routes between various nodes.
 ```python
graph.add_conditional_edges("node_1", routing_function)
```
The Routing function accepts current states of the graph, and returns a value. The default return value is used as the name of node or list of nodes to send the state to the next. Similar to functional programming chains.

You can also provide a dictionary of optional routes.
```python
graph.add_conditional_edges("node_a", routing_function, {True: "node_b", False: "node_c"})
```

In [5]:
import random
from typing import Literal

def decide_mode(state) -> Literal["node_2", "node_3"]:
    user_input = state['graph_state']
    if random.random() < 0.5:
        return "node_2"
    return "node_3"